In [1]:
!python main.py --model efficientnet_b4 --epochs 50 --wandb True


/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2023-04-20 12:23:29.128945: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### ignore below ------------------------------------------------------------------------------------------------


In [ ]:
'''
Plant Seedlings Classification project for Learning Theory Spring 23 NYU
Authors: Kevin Choi, ...

https://www.kaggle.com/competitions/plant-seedlings-classification
'''

#Import Statement

import torch
from torch import optim
from transformers import AdamW, AutoModelForMaskedLM, AutoConfig
from transformers import get_linear_schedule_with_warmup

import timm
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader, random_split
from torchmetrics.functional import accuracy, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt
import re
import os 
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MultiLabelBinarizer

from transformers import AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from torchmetrics import Accuracy

import pytorch_lightning as pl
# from torchvision import transforms
# import torchvision.transforms as tt
from torchvision import transforms

from torchvision.datasets import ImageFolder
from pytorch_lightning.loggers import WandbLogger
import wandb

import numpy as np
import pandas as pd
import random
import time
import datetime
import pickle 
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

from os import listdir
from os.path import isfile, join

In [4]:
!wandb login

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
wandb: Currently logged in as: kchoi1230. Use `wandb login --relogin` to force relogin


In [5]:
wandb.init(project="stat_final", entity="kchoi1230")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kchoi1230. Use `wandb login --relogin` to force relogin
2023-04-20 12:23:34.076827: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [6]:
#Data Prep. & Processsssss

class PlantDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.transform = transform
        self.df = dataframe
        self.root_dir = root_dir
        #self.classes = 
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        fullpath = os.path.join(self.root_dir, self.df.iloc[idx][0])
        image = Image.open(fullpath).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.df.iloc[idx][2]

class PlantDataModule(pl.LightningDataModule):
    def __init__(self, train_df, val_df, test_df, batch_size, data_dir: str = os.getcwd() + '/plant/data'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.num_classes = 12
        self.validation_split_ratio = 0.1
  
    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        
        self.train_tfms = transforms.Compose([
            transforms.RandomRotation(180),
            transforms.RandomAffine(degrees = 0, translate = (0.2, 0.2)),
            #transforms.CenterCrop(356),
            transforms.Resize((256,256)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.val_tfms = transforms.Compose([
            #transforms.CenterCrop(356),
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        # self.train_tfms = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224)), 
        #             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        # self.val_tfms = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224)),
        #             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        self.train_dataset = PlantDataset(self.train_df ,root_dir, self.train_tfms)
        self.val_dataset = PlantDataset(self.val_df ,root_dir, self.val_tfms)
        self.test_dataset = PlantDataset(self.test_df ,root_dir, self.val_tfms)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

class ImagePredictionLogger(pl.callbacks.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples
    
    def on_validation_epoch_end(self, trainer, pl_module):
        # Bring the tensors to CPU
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        # Get model prediction
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        # Log the images as wandb Image
        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
            })

class PlantClassifier(pl.LightningModule):
    def __init__(self,lr=1e-4,n_classes=12):
        super().__init__()
        self.lr=lr
        self.save_hyperparameters() 
        # self.model= timm.create_model('vit_base_patch16_224_in21k',pretrained=True,num_classes=n_classes)
        # self.model= timm.create_model('maxvit_base_tf_512',pretrained=True, num_classes=0)
        self.model= timm.create_model('maxxvit_rmlp_small_rw_256', pretrained=True)

        self.accuracy = Accuracy()
        
    def configure_optimizers(self):
        opt= optim.AdamW(self.model.parameters(),self.lr)
        return opt
    
    def training_step(self, dl, idx):
        x,y=dl
        z = self.model(x)
        loss = F.cross_entropy(z,y)
        
        preds = torch.argmax(z, dim=1)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def validation_step(self, dl, idx):
        x,y = dl
        z = self.model(x)
        loss = F.cross_entropy(z,y)
        
        preds = torch.argmax(z, dim=1)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('val_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def test_step(self, dl, idx):
        x,y = dl
        z = self.model(x)
        preds = torch.argmax(z, dim=1)
        # self.log("test_loss", loss, prog_bar=True, logger=True)
        return preds

    def predict_step(self, dl, idx, dataset_idx=0):
        x,y = dl
        z = self.model(x)
        preds = torch.argmax(z, dim=1)
        # self.log("predic_loss", loss, prog_bar=True, logger=True)
        return preds
    
def find_classes(fulldir):
    classes = os.listdir(fulldir)
    classes.sort()
    class_to_idx = dict(zip(classes, range(len(classes)))) 
    idx_to_class = {v:k for k,v in class_to_idx.items()}
    train = []
    
    for i, label in idx_to_class.items():
        path = fulldir+"/"+label
        for file in os.listdir(path):
            train.append([f'{label}/{file}', label, i])
    df = pd.DataFrame(train, columns=["file", "class", "class_index"]) 
    return classes, class_to_idx, idx_to_class, df

class TestPlant(Dataset):
    def __init__(self, rootdir, transform=None):
        self.transform=transform
        self.rootdir=rootdir
        self.image_files = os.listdir(self.rootdir)
    def __len__(self):
        return len(self.image_files)
    def __getitem__(self, idx):
        fullpath = os.path.join(self.rootdir, self.image_files[idx])
        image = Image.open(fullpath).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]
    
def load_preprocess_image(img_path, transforms):
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = transforms(img)
    return img

In [7]:
root_dir = os.getcwd() + "/plant/data/train"
# root_dir2 = os.getcwd() + "/plant/data/test"
classes, class_to_idx, idx_to_class, df = find_classes(root_dir)

In [8]:
root_dir = os.getcwd() + "/plant/data/train"
# root_dir2 = os.getcwd() + "/plant/data/test"
classes, class_to_idx, idx_to_class, df = find_classes(root_dir)
num_classes = len(classes)

df2 = pd.concat([df]*5, ignore_index=True)
df_train, df_val = train_test_split(df2,
                                    test_size=0.1, 
                                    random_state=1230, 
                                    stratify=df2['class_index'])

val_tfms = transforms.Compose([
            #transforms.CenterCrop(356),
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

test_dataset = TestPlant(rootdir = os.getcwd() + "/plant/data/test", transform=val_tfms)
test_loader = DataLoader(test_dataset, batch_size = 32, num_workers= 4, shuffle=False)

In [9]:
df_train.max()

file           Sugar beet/ffa401155.png
class                        Sugar beet
class_index                          11
dtype: object

In [10]:
df_val.min()

file           Black-grass/0260cffa8.png
class                        Black-grass
class_index                            0
dtype: object

In [11]:
df_test = pd.DataFrame(os.listdir(os.getcwd() + "/plant/data/test"))
df_test[1] = df_test.shape[0]*[0]
# df_test[2] = df_test.shape[0]*[0]
# df_test[0] = 'test/'+df_test[0]
df_test

,0,1
0,2406d6c99.png,0
1,9aa5587fe.png,0
2,089ad62a7.png,0
3,558aa7deb.png,0
4,caa2fbd79.png,0
...,...,...
789,115f93ecc.png,0
790,16357b436.png,0
791,664194d19.png,0
792,995c7ab1e.png,0


In [12]:
wandb_logger = WandbLogger(project='wandb-lightning', job_type='train')
# checkpoint_callback = pl.callbacks.ModelCheckpoint()
tmpdir = os.getcwd() + '/plant/model_weights/'
checkpoint_callback = ModelCheckpoint(dirpath=tmpdir, save_last=True, monitor="val_loss",filename='sample-mnist-{epoch:02d}-{val_loss:.2f}')

# Initialize a trainer
trainer = pl.Trainer(max_epochs=50,
                     progress_bar_refresh_rate=1, 
                     gpus=1, 
                     logger=wandb_logger,
                     checkpoint_callback=[checkpoint_callback])

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py:348: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=[<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7effafa07410>])` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=[<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7effafa07410>])`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/home/kchoi/.conda/envs/pl/lib/

In [13]:
model=PlantClassifier()
pdm = PlantDataModule(train_df = df2, val_df = df2.iloc[:10], test_df = df2.iloc[:10], batch_size=32)
pdm.setup()

# trainer= pl.Trainer(max_epochs=10)


/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [14]:
checkpoint_callback = ModelCheckpoint(dirpath=tmpdir, save_last=True, monitor="val_loss",filename='sample-mnist-{epoch:02d}-{val_loss:.2f}')

# Initialize a trainer
# trainer = pl.Trainer(max_epochs=50,
#                      progress_bar_refresh_rate=1, 
#                      gpus=1, 
#                      logger=wandb_logger,
#                      checkpoint_callback=[checkpoint_callback])

trainer.fit(model, pdm)
trainer.save_checkpoint("./plant/model_weights/last.ckpt")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type     | Params
--------------------------------------
0 | model    | MaxxVit  | 66.0 M
1 | accuracy | Accuracy | 0     
--------------------------------------
66.0 M    Trainable params
0         Non-trainable params
66.0 M    Total params
264.042   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: 0it [00:00, ?it/s]

In [15]:
test_result = trainer.predict(dataloaders=test_loader)
# pdm.test_dataloader()

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `predict(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /home2/kchoi/image/stat_final/38gafdhy/checkpoints/epoch=49-step=37150.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home2/kchoi/image/stat_final/38gafdhy/checkpoints/epoch=49-step=37150.ckpt


Predicting: 743it [00:00, ?it/s]

In [16]:
np.prod(np.array([32, 11, 7, 11, 7, 128]))

24285184

In [17]:
26873856/24285184

1.1065947039973014

In [18]:
test_result_list = np.hstack(pd.Series(test_result).apply(lambda x : x.tolist()))

In [19]:
df_test[1] = pd.Series(test_result_list).apply(lambda x : idx_to_class[x])
df_test

,0,1
0,2406d6c99.png,Shepherds Purse
1,9aa5587fe.png,Loose Silky-bent
2,089ad62a7.png,Common Chickweed
3,558aa7deb.png,Black-grass
4,caa2fbd79.png,Common Chickweed
...,...,...
789,115f93ecc.png,Loose Silky-bent
790,16357b436.png,Loose Silky-bent
791,664194d19.png,Small-flowered Cranesbill
792,995c7ab1e.png,Common wheat


In [20]:
df_test.columns = ['file','species']
df_test.to_csv('test_sample4.csv', index=False)

In [11]:
cd /home2/kchoi/image/plant

/home2/kchoi/image/plant


In [19]:
!python main.py --model efficientnet_b4 --epochs 50 --wandb True

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2023-04-22 04:56:13.822649: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
wandb: Currently logged in as: kchoi1230. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.0
wandb: Run data is saved locally in /home2/kchoi/image/plant/wandb/run-20230422_045617-yv5ksp4t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run wandering-gorge-55
wandb: ⭐️ View project at https://wandb.ai/kchoi1230/stat_final
wandb

In [23]:
pdm = PlantDataModule(train_df = df_val, val_df = df_val, test_df = df_val, batch_size=32)
pdm.setup()

trainer = pl.Trainer(max_epochs=55, resume_from_checkpoint="./plant/model_weights/last.ckpt")
trainer.fit(model, pdm)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at ./plant/model_weights/last.ckpt
/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:343: UserWarning: The dirpath has changed from '/home2/kchoi/image/stat_final/3nw0uexd/checkpoints' to '/home2/kchoi/image/lightning_logs/version_1/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
  f"The dirpath has changed from {dirpath_from_ckpt!r} to {self.dirpath!r},"

  | Name     | Type         | Params
------------------------------------------
0 | model    | EfficientNet | 17.5 M
1 | accuracy | Accuracy     | 0     
------------------------------------------
17.5 M    Trainable params
0         Non-trainable params
17.5 M    Total

Sanity Checking: 0it [00:00, ?it/s]

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 668it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [27]:
test_result = trainer.predict(dataloaders=test_loader)
# pdm.test_dataloader()
test_result_list = np.hstack(pd.Series(test_result).apply(lambda x : x.tolist()))
df_test['species'] = pd.Series(test_result_list).apply(lambda x : idx_to_class[x])
# df_test.columns = ['file','species']
# df_test.to_csv('test_sample2.csv', index=False)

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `predict(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /home2/kchoi/image/lightning_logs/version_1/checkpoints/epoch=54-step=33775.ckpt
Loaded model weights from checkpoint at /home2/kchoi/image/lightning_logs/version_1/checkpoints/epoch=54-step=33775.ckpt


Predicting: 75it [00:00, ?it/s]

In [33]:
df_test.iloc[:,:-1].to_csv('test_sample2.csv', index=False)

In [39]:
df_test[df_test['species'] != df_test2['species']]

,file,species,1
36,c6c8d4ba0.png,Black-grass,Black-grass
56,b0acaff4a.png,Loose Silky-bent,Loose Silky-bent
150,e82017baa.png,Black-grass,Black-grass
208,6a47821f9.png,Black-grass,Black-grass
265,b2706e2b3.png,Black-grass,Black-grass
486,fba8fc78a.png,Loose Silky-bent,Loose Silky-bent
681,d689256be.png,Black-grass,Black-grass
770,00d090cde.png,Black-grass,Black-grass


In [40]:
df_test2[df_test['species'] != df_test2['species']]

,file,species
36,c6c8d4ba0.png,Loose Silky-bent
56,b0acaff4a.png,Black-grass
150,e82017baa.png,Loose Silky-bent
208,6a47821f9.png,Loose Silky-bent
265,b2706e2b3.png,Common wheat
486,fba8fc78a.png,Black-grass
681,d689256be.png,Loose Silky-bent
770,00d090cde.png,Loose Silky-bent


In [32]:
df_test.to_csv('test_sample2.csv', index=False)

In [12]:
wandb.finish()

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


epoch,▁▁▃▃▃▆▆▆▆█
train_acc_epoch,█▁▁
train_acc_step,█▁▁▁
train_loss_epoch,█▁▁
train_loss_step,█▃▁▁
trainer/global_step,▃▁▁▁▁▁▁▁▁▃▃▄▁▁▁▁▁▁▁▂▆▆▆█▂▂▂▂▂▂▂▂██
val_acc_epoch,▅█▁
val_acc_step,▁█▁▃▁▃▁▁▁█▆▁▁▃▁▇▁▃▁▁▁▃▁▁
val_loss_epoch,█▃▁
val_loss_step,▅▇▆▅▃▇▃▄▄█▃▁▂▆▃▅▄▅▁▂▂▅▃▅
epoch,3


In [81]:
# test_data_dir = os.getcwd()+"/test"
device = 'cuda'
save_path = '/plant/test'
test_data_dir='/home2/kchoi/image/plant/data/train/test'
generate_predictions(model, test_data_dir, classes, device, save_path)

NotImplementedError: 

In [11]:
# trainer.test(dataloaders=
# )
val_samples = next(iter(pdm.test_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape

NameError: name 'pdm' is not defined

In [31]:
# trainer.test(dataloaders=
# )
val_samples = next(iter(test_loader))
val_imgs, val_labels = val_samples[0], val_samples[1]
# val_imgs.shape, val_labels.shape

In [33]:
val_labels

('2406d6c99.png',
 '9aa5587fe.png',
 '089ad62a7.png',
 '558aa7deb.png',
 'caa2fbd79.png',
 'cbe761896.png',
 '4b032563b.png',
 '3f92d8039.png',
 '0ad9e7dfb.png',
 '653193c1a.png',
 'e9d48d664.png',
 '780bd2a2c.png',
 'a7bd7cadb.png',
 'cadab6616.png',
 '13b9fa92d.png',
 '0d117d910.png',
 'bfab3e3d0.png',
 '8bc0261c9.png',
 '219fd68d5.png',
 '2f0004a7f.png',
 '4e69d100a.png',
 '39858776a.png',
 '6edb96d45.png',
 '837ac0270.png',
 'a8b431a3e.png',
 'cf90fc52d.png',
 '3f826b318.png',
 '7b21ba6ba.png',
 '711b46fba.png',
 '590f5aea6.png',
 '8e29abce1.png',
 '3f64c2c1b.png')

In [35]:
df_test.sort_values(by='file')

,file,species
151,0021e90e4.png,Small-flowered Cranesbill
537,003d61042.png,Fat Hen
299,007b3da8b.png,Sugar beet
650,0086a6340.png,Common Chickweed
718,00c47e980.png,Sugar beet
...,...,...
618,fea355851.png,Loose Silky-bent
655,fea3da57c.png,Sugar beet
560,fef2ade8c.png,Sugar beet
79,ff65bc002.png,Charlock


In [9]:
pretrained_model_list = timm.list_models(pretrained=True)

print(pretrained_model_list)


['adv_inception_v3', 'bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_224_in22k', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_224_in22k', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_base_patch16_224_in22k', 'beitv2_large_patch16_224', 'beitv2_large_patch16_224_in22k', 'botnet26t_256', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_tiny', 'coatnet_0_rw_224', 'coatnet_1_rw_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_rw_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_nano_rw_224', 'coatnext_nano_rw_224', 'convit_base', 'convit_small', 'convit_tiny', 'convmixer_768_32', 'convmixer_1024_20_ks9_p14', 'convmixer_1536_20', 'convnext_atto', 'convnext_atto_ols', 'convnext_base', 'convnext_base_384_in

In [11]:
!rm -r "/home2/kchoi/image/plant/data/train/Fat Hen/.ipynb_checkpoints"

In [27]:
!pwd

/home2/kchoi/image


In [12]:
%autosave 0

Autosave disabled


In [12]:
pdm = PlantDataModule(batch_size=64)
pdm.setup()

TypeError: __init__() missing 3 required positional arguments: 'train_df', 'val_df', and 'test_df'

In [16]:
val_samples = next(iter(pdm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

tensor([[[[ 0.1137,  0.1488,  0.2167,  ..., -0.2310, -0.1951, -0.1765],
          [ 0.1137,  0.1488,  0.2167,  ..., -0.2354, -0.2047, -0.1889],
          [ 0.1137,  0.1488,  0.2167,  ..., -0.2439, -0.2234, -0.2128],
          ...,
          [ 0.1611,  0.1652,  0.1732,  ...,  0.2160,  0.2183,  0.2196],
          [ 0.1092,  0.1133,  0.1213,  ...,  0.1505,  0.1671,  0.1757],
          [ 0.0824,  0.0865,  0.0945,  ...,  0.1166,  0.1405,  0.1529]],

         [[ 0.1216,  0.1629,  0.2427,  ..., -0.2370, -0.1971, -0.1765],
          [ 0.1216,  0.1629,  0.2427,  ..., -0.2394, -0.2047, -0.1868],
          [ 0.1216,  0.1629,  0.2427,  ..., -0.2439, -0.2194, -0.2068],
          ...,
          [ 0.1436,  0.1482,  0.1571,  ...,  0.1400,  0.1424,  0.1436],
          [ 0.0877,  0.0934,  0.1043,  ...,  0.0626,  0.0792,  0.0877],
          [ 0.0588,  0.0650,  0.0770,  ...,  0.0225,  0.0464,  0.0588]],

         [[ 0.1529,  0.1963,  0.2801,  ..., -0.2092, -0.1773, -0.1608],
          [ 0.1529,  0.1958,  

In [68]:
model = LitModel((3, 32, 32), pdm.num_classes)

# Initialize wandb logger
wandb_logger = WandbLogger(project='wandb-lightning', job_type='train')

# Initialize Callbacks
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_loss")
checkpoint_callback = pl.callbacks.ModelCheckpoint()

# Initialize a trainer
trainer = pl.Trainer(max_epochs=10,
                     gpus=1, 
                     logger=wandb_logger,
                     callbacks=[early_stop_callback,
                                ImagePredictionLogger(val_samples),
                                checkpoint_callback],
                     )

# Train the model ⚡🚅⚡
trainer.fit(model, pdm)

# Evaluate the model on the held-out test set ⚡⚡
trainer.test(dataloaders=dm.test_dataloader())

# Close wandb run
wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type      | Params
---------------------------------------
0 | conv1    | Conv2d    | 896   
1 | conv2    | Conv2d    | 9.2 K 
2 | conv3    | Conv2d    | 18.5 K
3 | conv4    | Conv2d    | 36.9 K
4 | pool1    | MaxPool2d | 0     
5 | pool2    | MaxPool2d | 0     
6 | fc1      | Linear    | 819 K 
7 | fc2      | Linear    | 65.7 K
8 | fc3      | Linear    | 1.5 K 
9 | accuracy | Accuracy  | 0     
---------------------------------------
952 K     Trainable params
0         Non-trainable params
952 K     Total params
3.810     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

NameError: name 'dm' is not defined

/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/home/kchoi/.conda/envs/pl/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/kchoi/.conda/envs/pl/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 152, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 138, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/home/kchoi/.conda/envs/pl/lib/python3.7/site-packages/wandb/sdk/interface/

In [6]:
df

,file,class,class_index
0,Black-grass/7050b0b8a.png,Black-grass,0
1,Black-grass/40bf7be90.png,Black-grass,0
2,Black-grass/dcbd3fa08.png,Black-grass,0
3,Black-grass/88fceea2f.png,Black-grass,0
4,Black-grass/89f06ca64.png,Black-grass,0
...,...,...,...
4747,Sugar beet/12c104178.png,Sugar beet,11
4748,Sugar beet/0b992e297.png,Sugar beet,11
4749,Sugar beet/5be7ccae6.png,Sugar beet,11
4750,Sugar beet/e7bed72a5.png,Sugar beet,11


In [29]:
#Model


import pytorch_lightning as pl
import torch
from torchmetrics.functional import auroc
from transformers import AdamW#, AutoModelForMaskedLM, AutoConfig
from transformers import get_linear_schedule_with_warmup
import torch.nn as nn
import pickle

from torchvision import transforms
from torchvision.datasets import CIFAR10

class Plant_Detection(pl.LightningModule):
    """
    Multi-class Classification model for Plant seedling Detection Task
    """
    def __init__(self, n_classes: int = 12,
                 n_training_steps=None,
                 n_warmup_steps=None,
                 config=None):
        """
        Args:
            :param n_classes: Number of classes to classify
            :param n_training_steps: Number of training steps, not used for inferencing
            :param n_warmup_steps: Number of Warmup steps: not used for inferencing
        """
        super().__init__()
        
        #TODO1: Edit to img model 
        # self.config = AutoConfig.from_pretrained("klue/roberta-base", output_hidden_states=True)
        # self.model = AutoModelForMaskedLM.from_pretrained("klue/roberta-base", config=self.config)
#         self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)
#         self.n_training_steps = n_training_steps
#         self.n_warmup_steps = n_warmup_steps
#         self.criterion = nn.BCEWithLogitsLoss()  # nn.BCELoss() with sigmoid layer
#         self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
#         self.dense = nn.Linear(self.model.config.hidden_size, self.model.config.hidden_size)
#         self.activation = nn.Tanh()
        

    def forward(self, input_ids, attention_mask=None, token_type_ids=None,
            position_ids=None, head_mask=None, labels=None):
        output = self.model(input_ids, attention_mask=attention_mask)
        last_hidden_state = output.hidden_states[-1]
        pooled_output = self.classifier(self.dropout(self.activation(self.dense(last_hidden_state[:, 0]))))
        loss = 0
        if labels is not None:
            loss = self.criterion(pooled_output, labels)
        return loss, torch.sigmoid(pooled_output)

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def predict_step(self, batch, batch_idx, dataset_idx=0):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # self.log("predic_loss", loss, prog_bar=True, logger=True)
        return loss, outputs

    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"]:
                labels.append(out_labels)
            for out_predictions in output["predictions"]:
                predictions.append(out_predictions)
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        for i, name in enumerate([0]*10):
            class_roc_auc = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-4)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.n_warmup_steps,
                                                    num_training_steps=self.n_training_steps)
        return dict(optimizer=optimizer, lr_scheduler=dict(scheduler=scheduler, interval='step'))
